<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/signate1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

9回目の提出さ

ここまでのtry
*   Light GBM、特徴量12個くらい。MinMaxScalerで正規化したけど、不必要かも
*   基準を0.1以上をOKとしたらクリアになった。？？？？怪しいな
*   crossvalidationがちゃんと働いてないな。
*　 cataegorical featuresという形でもうできることがわかったぞ


---


今後の課題
*   optunaを入れて調整しよう
*   過学習の判断をできるようにしよう
*   Minmaxとかの正規化ってcategoricaldataにもやっていいの？
*   
*   


In [1]:
#importまとめ。pandas-profilingは少々導入が面倒
!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas_profiling
import seaborn as sns
from pandas_profiling.utils.cache import cache_file
%matplotlib inline

warnings.filterwarnings('ignore')

  Cloning https://github.com/pandas-profiling/pandas-profiling.git to /tmp/pip-req-build-y39nhgmu
  Running command git clone -q https://github.com/pandas-profiling/pandas-profiling.git /tmp/pip-req-build-y39nhgmu
     |████████████████████████████████| 71kB 298kB/s 
     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 604kB 4.6MB/s 
     |████████████████████████████████| 3.1MB 7.6MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 296kB 36.2MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.9.0rc1-py2.py3-none-any.whl size=258106 sha256=4c395ad3d1884760306598af12d2e370471a280dc9e6a50450181ced89649db7
  Stored in directory: /tmp/pip-ephem-wheel-cache-7dfqw0d7/wheels/cd/13/75/8fcecd52c706914e90d916ede57f8c74de60e4c9ebc6c2f3b7
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=7de2cf4375b0d81debd0bd1228616fb11ce1745418183bfe07c6235b800418a2


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

sample_data


In [4]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

Saving submit_sample.csv to submit_sample.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [36]:
!ls

sample_data		       submit_sample.csv  train.csv
submission_lightgbm_kfold.csv  test.csv


In [111]:
#上手く列指定ができなかったので、先頭にheaderをつける
sub = pd.read_csv("submit_sample.csv",names=("A","B"))
#test,train,その足し合わせdataの三種を作る
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
data = pd.concat([train,test], sort=False)

#特徴量をエンジニアリング
data["job"].replace(['blue-collar','management','technician','admin.','services','unknown','self-employed','entrepreneur','student','retired','unemployed','housemaid'],[0,1,2,3,4,5,6,7,8,9,10,11],inplace=True)
data["loan"].replace(['yes','no'],[1,0],inplace=True)
data["marital"].replace(['married','single','divorced'],[2,1,0],inplace=True)
data['education'].replace(['secondary','tertiary','primary','unknown'],[3,2,1,0],inplace=True)
data["housing"].replace(['yes','no'],[1,0],inplace=True)
data["contact"].replace(['cellular','telephone','unknown'],[2,1,0],inplace=True)
data["poutcome"].replace(['success','failure','unknown','other'],[3,2,1,0],inplace=True)
data["job"] = data["job"].astype(np.int64)
data["month"].replace(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','des'],[3,4,5,6,7,8,9,10,11,0,1,2],inplace=True)

#全体のうち、残したいモノだけ選ぶ
all_columns = ['id','age','job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome']
remain_columns = ['age','job','marital','education','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome']
delete_columns = list(set(all_columns)-set(remain_columns))
data.drop(delete_columns, axis=1, inplace=True)

#trainとtestを再度切り分け
train = data[:len(train)]
test = data[len(train):]


**------------------cross validation--------------------**


**------light GBM-------**

In [112]:
#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。三種類がでる
y_train = train['y']
X_train = train.drop('y', axis = 1)
X_test = test.drop('y', axis = 1)

# MinmaxScalarを使って正規化。
from sklearn.preprocessing import MinMaxScaler
mmsc = MinMaxScaler()
# 訓練用のデータを正規化する。fit_transformはfit＆transformの略で、情報取得＆正規化の意味
X_train_norm = mmsc.fit_transform(X_train)
# 訓練用データを基準にテストデータも正規化
X_test_norm = mmsc.transform(X_test)
X_train=X_train_norm
X_test=X_test_norm
#正規化するとarrayになっていたので、DataFrameに戻す
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
#Dataflameにしたらheaderが抜けていたので付け直す
X_train.columns=remain_columns
X_test.columns=remain_columns
#この辺のheader周りの操作は無駄なのでできたら修正したい

In [113]:
#欲しいデータが見えているか確認
data

,age,job,marital,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,31,4,2,3,12294,1,0,2,21,1,101,3,498,0,0,0.0
1,29,7,1,2,43027,0,0,2,22,10,158,2,702,0,1,1.0
2,35,1,2,2,12252,1,0,2,11,1,351,1,826,0,2,0.0
3,31,2,2,3,99121,1,1,0,16,7,658,2,120,0,2,0.0
4,48,10,2,1,42005,1,0,1,3,6,177,1,273,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,49,6,2,2,98357,1,0,2,6,9,101,2,417,0,2,NaN
18046,34,0,2,3,29621,1,0,2,12,7,345,1,815,0,1,NaN
18047,34,3,1,3,94260,1,0,0,16,7,121,2,370,0,1,NaN
18048,31,2,1,3,65483,1,0,0,15,7,345,2,41,0,1,NaN


In [114]:
remain_columns

['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome']

In [116]:
X_train

,age,job,marital,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,0.132353,0.363636,1.0,1.000000,0.175657,1.0,0.0,1.0,0.666667,0.090909,0.032835,0.50,0.572905,0.0,0.000000
1,0.102941,0.636364,0.5,0.666667,0.457694,0.0,0.0,1.0,0.700000,0.909091,0.051365,0.25,0.807118,0.0,0.333333
2,0.191176,0.090909,1.0,0.666667,0.175272,1.0,0.0,1.0,0.333333,0.090909,0.114109,0.00,0.949483,0.0,0.666667
3,0.132353,0.181818,1.0,1.000000,0.972469,1.0,1.0,0.0,0.500000,0.636364,0.213914,0.25,0.138921,0.0,0.666667
4,0.382353,0.909091,1.0,0.333333,0.448315,1.0,0.0,0.5,0.066667,0.545455,0.057542,0.00,0.314581,0.0,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27095,0.220588,0.000000,1.0,1.000000,0.307503,1.0,0.0,1.0,0.866667,0.636364,0.112159,0.75,0.489093,0.0,0.333333
27096,0.191176,0.363636,1.0,1.000000,0.449646,1.0,0.0,1.0,0.866667,0.636364,0.039337,0.00,0.826636,0.0,0.333333
27097,0.191176,0.363636,1.0,0.000000,0.379726,0.0,0.0,1.0,0.900000,0.727273,0.057542,0.25,0.140069,0.0,0.333333
27098,0.117647,0.272727,0.5,1.000000,0.977057,1.0,0.0,1.0,0.866667,0.636364,0.039337,0.00,0.115959,0.0,0.333333


In [118]:
#LightGBMに入れる。
from sklearn.model_selection import KFold
import lightgbm as lgb

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

#categorical dataの入れ忘れに注意
categorical_features = ['job','marital','education','housing','loan','contact','duration','poutcome']

params = {
    'objective': 'binary',
    'max_bin': 400,
    'learning_rate': 0.01,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,
                                            categorical_feature=categorical_features)
    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)
    
    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.263387	valid_1's binary_logloss: 0.241708
[20]	training's binary_logloss: 0.253434	valid_1's binary_logloss: 0.234226
[30]	training's binary_logloss: 0.245986	valid_1's binary_logloss: 0.228869
[40]	training's binary_logloss: 0.240084	valid_1's binary_logloss: 0.224508
[50]	training's binary_logloss: 0.235203	valid_1's binary_logloss: 0.221101
[60]	training's binary_logloss: 0.231103	valid_1's binary_logloss: 0.218341
[70]	training's binary_logloss: 0.227643	valid_1's binary_logloss: 0.216104
[80]	training's binary_logloss: 0.224652	valid_1's binary_logloss: 0.214072
[90]	training's binary_logloss: 0.222064	valid_1's binary_logloss: 0.21242
[100]	training's binary_logloss: 0.219718	valid_1's binary_logloss: 0.210905
[110]	training's binary_logloss: 0.217632	valid_1's binary_logloss: 0.209627
[120]	training's binary_logloss: 0.215638	valid_1's binary_logloss: 0.208611
[130]	training's binary_

In [119]:
#pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

===CV scores===
[0.199560412325057, 0.21790085931723194, 0.21885933713711517, 0.21348025247365507, 0.2146640610607974]
0.21289298446277133


In [124]:
from sklearn.metrics import accuracy_score
percentage=0.05

y_pred_oof = (oof_train > percentage).astype(int)
accuracy_score(y_train, y_pred_oof)

0.6171955719557195

In [125]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > percentage).astype(int)


sub['B'] = y_sub
sub.to_csv('submission_lightgbm_kfold.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

(8011, 18050)

In [122]:
sub

,A,B
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
18045,18045,0
18046,18046,0
18047,18047,0
18048,18048,0


-----ここまでcross validation-------

In [126]:
files.download('submission_lightgbm_kfold.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>